# Tavily地图

[Tavily](https://tavily.com/) 是专为AI代理（LLMs）构建的搜索引擎，能够快速提供实时、准确且基于事实的结果。Tavily提供了四个主要端点，其中之一是“地图”，它以图的方式遍历网站，可以通过智能发现并行探索数百条路径，从而生成全面的站点地图。

本指南简要概述了如何开始使用Tavily [工具](/docs/integrations/tools/)。如需对Tavily工具进行详细解读，您可以在[API参考文档](https://v03.api.js.langchain.com/modules/_langchain_tavily.html)中找到更多信息。

## 概览

### 集成详情

| 类别 | 包 | [Python支持](https://python.langchain.com/docs/integrations/tools/tavily_search/) | 最新包 |
| :--- | :--- | :---: | :---: |
| [TavilyMap](https://api.js.langchain.com/classes/_langchain_tavily.TavilyMap.html) | [`@langchain/tavily`](https://www.npmjs.com/package/@langchain/tavily) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/tavily?style=flat-square&label=%20&) |

## 安装设置

该集成位于 `@langchain/tavily` 包中，您可以按如下方式安装：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/tavily @langchain/core
</Npm2Yarn>
```

### 凭证

在此处[申请](https://app.tavily.com)一个API密钥，并将其设置为名为 `TAVILY_API_KEY` 的环境变量。

```typescript
process.env.TAVILY_API_KEY = "YOUR_API_KEY"
```

此外，设置 [LangSmith](https://smith.langchain.com/) 以获得最佳的可观察性（非必需，但推荐）：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

你可以通过以下方式导入并实例化 `TavilyMap` 工具：

In [1]:
import { TavilyMap } from "@langchain/tavily";

const tool = new TavilyMap({
  maxDepth: 3,
  maxBreadth: 50,
  // limit: 100,
  // allowExternal: false,
});

## 调用

### [使用参数直接调用](/docs/concepts/tools)

Tavily 地图工具在调用时接受以下参数：

* `url`（必填）：一个自然语言搜索查询

* 调用期间还可以设置以下参数：`instructions`、`selectPaths`、`selectDomains`、`excludePaths`、`excludeDomains`、`allowExternal`、`categories`。

In [ ]:
await tool.invoke({
  url: "https://docs.tavily.com"
});

### [使用工具调用调用](/docs/concepts/tools)

我们还可以使用模型生成的 `ToolCall` 来调用工具，在这种情况下将返回一个 `ToolMessage`：

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    url: "https://docs.tavily.com"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

## 链式调用

我们可以通过首先将工具绑定到一个[工具调用模型](/docs/how_to/tool_calling/)，然后调用它，从而以链式方式使用我们的工具：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
})

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("https://docs.tavily.com");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## 代理

关于如何在代理中使用LangChain工具的指南，请参阅 [LangGraph.js](https://langchain-ai.github.io/langgraphjs/how-tos/#tool-calling) 文档。

## API 参考文档

如需详细了解所有 Tavily 地图 API 的功能和配置，请前往 API 参考文档：

https://docs.tavily.com/documentation/api-reference/endpoint/map

## 相关内容

* 工具[概念指南](https://js.langchain.com/docs/concepts/tools/)

* 工具[操作指南](https://js.langchain.com/docs/how_to/#tools)